### 遗传算法（Genetic Algorithm, GA）

#### 一、定义
遗传算法（GA）是一种**模拟生物进化过程的元启发式优化算法**，通过维护一个候选解种群，利用**选择（Selection）、交叉（Crossover）和变异（Mutation）** 三大遗传操作，在解空间中迭代搜索近似最优解。其核心思想是“**优胜劣汰，适者生存**”：适应度高的个体有更高概率将其基因传递给下一代，逐步提升种群整体质量。

> **关键特征**：
> - **种群基础（Population-based）**：并行探索多个解，避免陷入局部最优；
> - **无梯度依赖（Gradient-free）**：仅需目标函数的输入-输出映射（黑盒优化）；
> - **随机性与确定性结合**：选择具方向性（偏好高适应度），交叉/变异引入随机探索。

---

#### 二、基本流程（5 步循环）

GA 的完整迭代过程如下：

##### 1. **初始化（Initialization）**
- 随机生成大小为 *N* 的初始种群 $ P_0 = \{x_1, x_2, ..., x_N\} $；
- 每个个体 $ x_i $ 是问题解的编码表示（如二进制串、实数向量）；
- **编码方式决定搜索空间结构**：
  - 二进制编码：经典形式，适合离散问题；
  - 实数编码：直接用浮点数表示参数，适合连续优化（如神经网络权重）；
  - 排列编码：用于 TSP 等组合优化。

##### 2. **评估（Evaluation）**
- 对每个个体 $ x_i $，计算其**适应度（Fitness）**：  
  $ f_i = \text{fitness}(x_i) $
- 适应度函数即**目标函数本身或其单调变换**（如最大化问题直接用 $ f(x) $，最小化问题用 $ \frac{1}{1 + f(x)} $）；
- 此步骤是 GA 与环境/问题的唯一接口，**不依赖内部结构信息**。

##### 3. **选择（Selection）**
- 依据适应度从当前种群中选择个体作为**父代（Parents）**，用于生成下一代；
- **核心原则**：适应度越高，被选中概率越大；
- 常用策略：
  - **轮盘赌选择（Roulette Wheel）**：选择概率 $ p_i = \frac{f_i}{\sum_j f_j} $；
  - **锦标赛选择（Tournament）**：随机选 *k* 个个体，取其中最优者（更鲁棒，抗适应度尺度影响）；
  - **精英保留（Elitism）**：直接复制 top-*e* 个体到下一代，保证最优解不丢失。

##### 4. **交叉（Crossover / Recombination）**
- 对选中的父代配对，以概率 $ p_c $（交叉率）进行**基因重组**，生成子代；
- **目的**：组合双亲优良基因，探索新区域；
- 常用算子：
  | 编码类型 | 交叉算子 | 示例 |
  |---------|----------|------|
  | 二进制 | 单点交叉 | `[1 0|1 1 0] + [0 1|0 0 1] → [1 0 0 0 1], [0 1 1 1 0]` |
  | 实数 | 模拟二进制交叉（SBX） | $ c_1 = 0.5[(1+\beta)x_1 + (1-\beta)x_2] $（$\beta$ 随机） |
  | 实数 | 中间交叉 | $ c = \alpha x_1 + (1-\alpha)x_2 $ |

##### 5. **变异（Mutation）**
- 对子代个体，以低概率 $ p_m $（变异率）**随机扰动某些基因位**；
- **目的**：维持种群多样性，防止早熟收敛，跳出局部最优；
- 常用算子：
  - 二进制：**位翻转**（0 ↔ 1）；
  - 实数：**高斯变异** $ x_i \leftarrow x_i + \mathcal{N}(0, \sigma) $；
  - 实数：**多项式变异**（在边界内扰动）。

完成以上 5 步后，用新生成的子代替换旧种群（或结合精英），进入下一代迭代，直至满足终止条件（最大代数、适应度阈值、收敛判据等）。

---

#### 三、主要分支（按问题类型与机制演进）

GA 已发展出多个重要分支，适应不同优化场景：

##### ▶ 1. **按优化目标数划分**
| 分支 | 特点 | 代表算法 |
|------|------|---------|
| **单目标 GA** | 优化单一目标函数 | 经典 Holland GA, 实数编码 GA |
| **多目标 GA（MOGA）** | 同时优化多个冲突目标，输出 Pareto 最优解集 | NSGA, NSGA-II, **NSGA-III**, MOEA/D, SPEA2 |

---

##### ▶ 2. **按编码与搜索空间划分**
| 分支 | 适用问题 | 关键技术 |
|------|---------|---------|
| **二进制 GA** | 组合优化、离散问题 | 位串编码、单点/均匀交叉 |
| **实数 GA** | 连续参数优化（如神经网络训练） | SBX 交叉、多项式变异 |
| **遗传编程（GP）** | 符号回归、程序合成 | 树结构编码、子树交叉/变异 |
| **进化策略（ES）** | 连续黑箱优化 | （μ, λ）选择、自适应 σ 变异（如 CMA-ES） |

---

##### ▶ 3. **按应用领域特化**
| 分支 | 应用场景 | 特性 |
|------|---------|------|
| **NeuroEvolution** | 用 GA 优化神经网络（结构/权重） | 染色体 = 网络参数向量 或 拓扑描述 |
| **Hybrid GA** | GA + 局部搜索（如梯度下降） | GA 全局探索 + 局部优化精细搜索 |
| **约束 GA** | 含约束的优化问题 | 罚函数法、可行性规则、修复算子 |